In [ ]:
import pandas as pd
from mair.pdf_parsing import parse
from glob import glob
from tqdm import tqdm
import os
import json
import spacy
from collections import defaultdict

In [ ]:
nlp=spacy.load('en_core_web_sm')
nlp.max_length=3000000
tqdm.pandas()


In [ ]:
df = pd.read_csv("../data/oecd_meta.csv")
MODAL_VERBS = [
    "can",
    "could",
    "may",
    "might",
    "will",
    "would",
    "shall",
    "should",
    "must",
]

In [ ]:
paths = glob("../data/oecd_docs/*.pdf")
# filenames =[path.split('/')[-1] for path in paths]

In [ ]:
results = {}
for path in tqdm(paths):
    try:
        results[os.path.basename(path)] = parse(path).full_text
    except Exception as e:
        print(f"error parsing {path} ({e}), passing...")

In [ ]:
json.dump(results, open("../data/oecd_docs_texts.json", "w"))

In [ ]:
raw_texts = pd.Series(results)

In [ ]:
docs = raw_texts.progress_apply(nlp)

In [ ]:
# modal_sentences = defaultdict(list)
results = []
for id, doc in docs.iteritems():
    for t in doc:
        verb = t.lemma_.lower()
        if verb in MODAL_VERBS:
            a=t
            result = {
                "verb": verb,
                "sent": t.sent.text.replace('\n', ' '),
                "doc_id": id,
                'ancestor': next(t.ancestors, '')
            }
            results.append(result)

In [ ]:
result_df = pd.DataFrame(results)

In [ ]:
result_df.to_csv('../data/processed/modals_sentences.csv', index=False)